In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

df = pd.read_csv('../input/fond-treyd/train (2).csv', parse_dates=['<DATE>'], infer_datetime_format=True, index_col=2)
df.head()

In [ ]:
df.columns

In [ ]:
df.drop(['<TICKER>', '<PER>', '<TIME>', '<VOL>'], axis=1, inplace=True)

In [ ]:
df.tail()

In [ ]:
# !conda install -y conda-forge ta-lib 

In [ ]:
# !conda install TA-Lib

In [ ]:
# Technical Indicators
import talib as ta

# Plotting graphs
import matplotlib.pyplot as plt

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score


In [ ]:
df['S_10'] = df['<CLOSE>'].rolling(window=10).mean()
df['Corr'] = df['<CLOSE>'].rolling(window=10).corr(df['S_10'])
df['RSI'] = ta.RSI(np.array(df['<CLOSE>']), timeperiod =10)
df['Open-Close'] = df['<OPEN>'] - df['<CLOSE>'].shift(1)
df['Open-Open'] = df['<OPEN>'] - df['<OPEN>'].shift(1)
df = df.dropna()
X = df.iloc[:,:9]

In [ ]:
y = np.where(df['<CLOSE>'].shift(-1) > df['<CLOSE>'],-1,1)

In [ ]:
split = int(0.7*len(df))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

In [ ]:
model = LogisticRegression()
model = model.fit (X_train,y_train)

In [ ]:
# The Coefficients
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

In [ ]:
probability = model.predict_proba(X_test)
print(probability)

predicted = model.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, predicted))

In [ ]:
print(metrics.classification_report(y_test, predicted))

In [ ]:
print(model.score(X_test,y_test))

In [ ]:
cross_val = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
print(cross_val)
print(cross_val.mean())

In [ ]:
df['Predicted_Signal'] = model.predict(X)
df['Nifty_returns'] = np.log(df['<CLOSE>']/df['<CLOSE>'].shift(1))
Cumulative_Nifty_returns = np.cumsum(df[split:]['Nifty_returns'])

df['Startegy_returns'] = df['Nifty_returns']* df['Predicted_Signal'].shift(1)
Cumulative_Strategy_returns = np.cumsum(df[split:]['Startegy_returns'])

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(Cumulative_Nifty_returns, color='r',label = 'Nifty Returns')
plt.plot(Cumulative_Strategy_returns, color='g', label = 'Strategy Returns')
plt.legend()
plt.show()